neutral -> 0, anger -> 1,  joy -> 2  surprise -> 3, sadness -> 4, disgust -> 5, fear -> 6

'neutral': 0, 'joy': 1, 'anger': 2, 'surprise': 3, 'sadness': 4

In [1]:
#try preprocess V
#try embedding (need modify model)
#try attention
#try other component: decision tree

from torchtext.legacy import data
from torchtext import datasets
import torch

device = 'cuda'
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

fields = [('u', TEXT), (None, None), ('e',LABEL)]

train_data, valid_data = data.TabularDataset.splits(
    path = '',
    train = 'train_Aug_round2.csv',
    validation = 'dev_HW2dataset.csv',
    format = 'csv',
    fields = fields,
    skip_header = True
)

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = 64,
    device = device,
    sort_key = lambda x: len(x.u),
    sort_within_batch = True
)

print(vars(train_data.examples[0]))

C:\Users\jerry\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'u': ['Hey', ',', 'Mel', 'Gibson', 'and', 'Clint', 'Eastwood', '.'], 'e': 'surprise'}


In [2]:
from torchtext.vocab import GloVe
TEXT.build_vocab(train_data, max_size = 25000,vectors = GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.attention = Attention(embedding_dim, embedding_dim)#change next to attention_dim
        self.rnn = nn.GRU(embedding_dim, hidden_dim,num_layers = 2,bidirectional=True)
        self.fc = nn.Linear(2*hidden_dim, output_dim)
        
    def forward(self, x):  
        x2 = self.embedding(x)
        #x3 = self.attention(x2)
        output, hidden = self.rnn(x2)
        return self.fc(output[-1,:,:])
    
    
class Attention(nn.Module):
    #https://youtu.be/lqCAfu6GI2c
    def __init__(self,embedding_dim,attention_dim):
        super(Attention,self).__init__()
        self.embedding_dim = embedding_dim
        self.attention_dim = attention_dim
        self.query = nn.Linear(embedding_dim,attention_dim,bias = False)
        self.key = nn.Linear(embedding_dim,attention_dim,bias = False)
        self.value = nn.Linear(embedding_dim,attention_dim,bias = False)
    def forward(self, inputs):
        q = self.query(inputs)
        k = self.query(inputs)
        v = self.value(inputs)
        
        attn_score = torch.matmul(q,k.transpose(-1,-2))
        softmax_score = F.softmax(attn_score,dim=-1)
        output = torch.matmul(softmax_score,v)

        return output
    



In [4]:
#try 20 30 40 50 normal: epochs 35 best bidirectional:try 20 if bad giveup bidir
#glove  1e1 25
#try train r1, merge r1
#選一個合理的跟一個分數好的

#lr 2e-2 batch2 3 to 4 epoch (try 1e-2: still 4) 
from tqdm import trange
import torch.optim as optim

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 200
OUTPUT_DIM = 7
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model.embedding.weight.requires_grad)

model = model.to(device)

model.embedding.weight.data = TEXT.vocab.vectors.cuda()

optimizer = optim.SGD(model.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

criterion = criterion.to(device)

epoch = 20

maxacc = 0

for i in range (epoch):
    print('epoch '+str(i+1))
    epoch_loss = 0
    train_accuracy=0.0
    valid_accuracy=0.0
    model.train()
    for batch in train_iterator:
        optimizer.zero_grad()
        predictions = model(batch.u)#.squeeze(1)
        loss = criterion(predictions, batch.e.long())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _,prediction=torch.max(predictions.data,1)
        train_accuracy+=int(torch.sum(prediction==batch.e.data))
    print('loss:')
    print(epoch_loss / len(train_data))
    print('acc:')
    print(train_accuracy/len(train_data))
    model.eval()
    with torch.no_grad():
        for batch in valid_iterator:
            predictions = model(batch.u)#.squeeze(1)
            _,prediction=torch.max(predictions.data,1)
            valid_accuracy+=int(torch.sum(prediction==batch.e.data))
    print('val acc:')
    print(valid_accuracy/len(valid_data))
    if valid_accuracy>maxacc:
        maxacc = valid_accuracy
        torch.save(model,'GRU_lr11_attention.pt')
    print()

True
epoch 1
loss:
0.024174485797139236
acc:
0.48019986924441954
val acc:
0.48632010943912446

epoch 2
loss:
0.02312136915002404
acc:
0.49939292051928647
val acc:
0.49247606019151846

epoch 3
loss:
0.02256645472252125
acc:
0.5075651442981227
val acc:
0.5102599179206566

epoch 4
loss:
0.021890157732273714
acc:
0.5199869244419538
val acc:
0.5034199726402189

epoch 5
loss:
0.02142673627027802
acc:
0.5233492107966751
val acc:
0.5143638850889193

epoch 6
loss:
0.02090424420041531
acc:
0.5363780704212198
val acc:
0.5006839945280438

epoch 7
loss:
0.020412739228164604
acc:
0.5480526758195573
val acc:
0.5205198358413132

epoch 8
loss:
0.01999485335720123
acc:
0.5518352479686187
val acc:
0.493844049247606

epoch 9
loss:
0.01945228806628203
acc:
0.5647707107499766
val acc:
0.5259917920656635

epoch 10
loss:
0.018964203911868868
acc:
0.5762118240403474
val acc:
0.518467852257182

epoch 11
loss:
0.018524373753393487
acc:
0.5873260483795648
val acc:
0.5239398084815321

epoch 12
loss:
0.017893894233

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()




In [10]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neutral': 0, 'joy': 1, 'anger': 2, 'surprise': 3, 'sadness': 4, 'disgust': 5, 'fear': 6})


In [11]:
import numpy as np
import pandas as pd

model = torch.load('GRU_lr11_attention.pt')
predict = []

data = "test_HW2dataset.csv"
df = pd.read_csv(data,sep=',')

for index,row in df.iterrows():
    sentence = row['Utterance']
    predict.append(predict_class(model,str(sentence)))

for i in range(len(predict)):
    if predict[i] == 1:
        predict[i] =2
    elif predict[i] == 2:
        predict[i] = 1
        
print(predict)

[0, 3, 0, 0, 0, 0, 2, 3, 2, 2, 2, 2, 0, 0, 0, 4, 0, 0, 0, 3, 3, 0, 2, 3, 3, 3, 5, 5, 1, 1, 0, 1, 3, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 4, 0, 3, 2, 0, 3, 3, 3, 3, 0, 0, 3, 2, 5, 5, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 5, 1, 2, 2, 0, 0, 0, 0, 0, 2, 5, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 3, 0, 3, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 3, 3, 0, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 3, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 5, 0, 0, 0, 1, 2, 0, 2, 3, 5, 0, 2, 3, 6, 1, 6, 0, 3, 3, 6, 1, 6, 0, 3, 2, 3, 0, 0, 3, 5, 0, 3, 5, 0, 3, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 0, 0, 1, 0, 0, 0, 6, 1, 6, 0, 1, 2, 6, 1, 6, 0, 0, 0, 0, 0, 0, 1, 5, 0, 2, 4, 0, 0, 0, 2, 0, 0, 5, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 3, 0, 3, 0, 1, 5, 0, 2, 0, 0, 1, 5, 3, 2, 3, 6, 0, 2, 0, 3, 0, 2, 0, 1, 3, 4, 4, 0, 3, 3, 0, 5, 1, 2, 3, 

In [12]:
import csv
with open('lasr.csv','a',newline = '') as fd:
    writer = csv.writer(fd)
    writer.writerow(['index','emotion'])
    
    
for index,pred in enumerate(predict):
    with open('lasr.csv','a',newline = '') as fd:
        writer = csv.writer(fd)
        writer.writerow([index,pred])

In [11]:
df1 = pd.read_csv('train_for_rf.csv',sep = ',')
df1

,Utterance,Speaker,predict,Dialogue_ID,Utterance_ID,Old_Dialogue_ID,Old_Utterance_ID,Season,Episode,StartTime,EndTime,Emotion
0,also I was the point person on my company’s tr...,Chandler,0,0,0,0,0,8,21,"00:16:16,059","00:16:21,731",0
1,You must’ve had your hands full.,The Interviewer,0,0,1,0,1,8,21,"00:16:21,940","00:16:23,442",0
2,That I did. That I did.,Chandler,0,0,2,0,2,8,21,"00:16:23,442","00:16:26,389",0
3,So let’s talk a little bit about your duties.,The Interviewer,0,0,3,0,3,8,21,"00:16:26,820","00:16:29,572",0
4,My duties? All right.,Chandler,0,0,4,0,4,8,21,"00:16:34,452","00:16:40,917",3
...,...,...,...,...,...,...,...,...,...,...,...,...
12835,"When I first moved to the city, I went out a c...",Joey,5,2159,2,1038,11,2,3,"00:00:35,744","00:00:44,334",5
12836,It made me nuts.,Joey,5,2159,3,1038,12,2,3,"00:00:45,462","00:00:46,587",5
12837,"You guys are messing with me, right?",Joey,3,2159,4,1038,15,2,3,"00:01:00,518","00:01:03,520",3
12838,Yeah.,All,0,2159,5,1038,16,2,3,"00:01:05,398","00:01:07,274",0


In [31]:
import pandas as pd
predict = []
model = torch.load('GRU_lr11.pt').to(device)

data = "dev_HW2dataset.csv"
df = pd.read_csv(data,sep=',')

for index,row in df.iterrows():
    sentence = row['Utterance']
    predict.append(predict_class(model,str(sentence)))

for i in range(len(predict)):
    if predict[i] == 1:
        predict[i] =2
    elif predict[i] == 2:
        predict[i] = 1

df['predict'] = predict
i = list(df.columns)
a, b = i.index('Emotion'), i.index("predict")
i[b], i[a] = i[a], i[b]
df = df[i]

#neutral -> 0, anger -> 1, joy -> 2 surprise -> 3, sadness -> 4, disgust -> 5, fear -> 6
label_dict = {
    'neutral' : 0,
    'anger' : 1,
    'joy' : 2,
    'surprise' : 3,
    'sadness' : 4,
    'disgust' : 5,
    'fear' :6
}
newrow = []
for index,row in df.iterrows():
    row['Emotion'] = label_dict[row['Emotion']] 
    newrow.append(row['Emotion'])
    
df['Emotion'] = newrow
df.to_csv('dev_for_rf.csv',index = False)
df

ValueError: 'Emotion' is not in list

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score,accuracy_score
from sklearn.metrics import precision_score

df1 = pd.read_csv('train_for_rf.csv',sep = ',')
df2 = pd.read_csv('dev_for_rf.csv',sep = ',')
df3 = pd.read_csv('test_for_rf.csv',sep = ',')
df = pd.concat([df1,df2])
df = pd.concat([df,df3])

df = df.drop(columns = 'StartTime')
df = df.drop(columns = 'EndTime')
df = df.drop(columns = 'Utterance')
df = df.drop(columns = 'Dialogue_ID')
df = df.drop(columns = 'Old_Dialogue_ID')
df = df.drop(columns = 'Utterance_ID')
df = df.drop(columns = 'Old_Utterance_ID')
df = df.drop(columns = 'Season')
df = df.drop(columns = 'Episode')

features=df.iloc[:,0:2]
target=df.iloc[:,2]


dum = pd.get_dummies(features)
features = pd.DataFrame(dum)




#normalize

f1 = features.iloc[:12840,:]
f2 = features.iloc[12840:14302,:]
t1 = target.iloc[:12840]
t2 = target.iloc[12840:14302]
f3 = features.iloc[14302:,:]
t3 = target.iloc[14302:]
print(len(f1),len(f2),len(f3))
rf=RandomForestClassifier(n_estimators = 200,random_state=1)
rf.fit(f1, t1)


12840 1462 3400


RandomForestClassifier(n_estimators=200, random_state=1)

In [65]:

#features=df_2.iloc[:,0:8]
#target=df_2.iloc[:,8]

features = f2
target = t2

ypredict=rf.predict(features)
#print(sum(ypredict == target))
print("accuracy:",rf.score(features, target))
print("confusion matrix:")
print(confusion_matrix(ypredict, target))
print("precision score of each label",precision_score(target,ypredict,average=None))
print("recall score of each label",recall_score(target,ypredict,average=None))

accuracy: 0.5266757865937073
confusion matrix:
[[492  66  54  39  68  18   9]
 [  7  46  30  17  14   5  14]
 [ 32  55 102  32  28   2   7]
 [ 41  32  24 114  13   5  16]
 [  9   2   1   2   8   0   1]
 [  6   7   1   4   4   1   0]
 [ 13   0   2   2  10   0   7]]
precision score of each label [0.65951743 0.34586466 0.39534884 0.46530612 0.34782609 0.04347826
 0.20588235]
recall score of each label [0.82       0.22115385 0.47663551 0.54285714 0.05517241 0.03225806
 0.12962963]


In [67]:
features = f3
target = t3

ypredict=rf.predict(features)


import csv
with open('rf2.csv','a',newline = '') as fd:
    writer = csv.writer(fd)
    writer.writerow(['index','emotion'])
    
    
for index,pred in enumerate(predict):
    with open('rf2.csv','a',newline = '') as fd:
        writer = csv.writer(fd)
        writer.writerow([index,pred])

In [32]:
import pandas as pd
predict = []
model = torch.load('GRU_lr11.pt').to(device)

data = "test_HW2dataset.csv"
df = pd.read_csv(data,sep=',')

for index,row in df.iterrows():
    sentence = row['Utterance']
    predict.append(predict_class(model,str(sentence)))

for i in range(len(predict)):
    if predict[i] == 1:
        predict[i] =2
    elif predict[i] == 2:
        predict[i] = 1

df.insert(2,'predict',predict)
df.to_csv('test_for_rf.csv',index = False)
df

,Utterance,Speaker,predict,Dialogue_ID,Utterance_ID,Old_Dialogue_ID,Old_Utterance_ID,Season,Episode,StartTime,EndTime
0,Why do all you’re coffee mugs have numbers on ...,Mark,0,0,0,0,0,3,19,"00:14:38,127","00:14:40,378"
1,Oh. That’s so Monica can keep track. That way ...,Rachel,3,0,1,0,1,3,19,"00:14:40,629","00:14:47,385"
2,Y'know what?,Rachel,0,0,2,0,2,3,19,"00:14:56,353","00:14:57,520"
3,Okay.,Ross,0,1,0,2,0,5,16,"00:07:26,446","00:07:27,487"
4,"Ross, didn't you say that there was an elevato...",Rachel,3,1,1,2,1,5,16,"00:07:27,655","00:07:30,574"
...,...,...,...,...,...,...,...,...,...,...,...
3395,You just wanna each take one?,Monica,0,576,1,279,5,6,4,"00:14:13,269","00:14:14,644"
3396,Why don’t you phase it out?,Monica,3,576,2,279,9,6,4,"0:14:28,784","0:14:31,818"
3397,"Yeah, fade the accent out and people will thin...",Monica,0,576,3,279,10,6,4,"00:14:28,200","00:14:35,039"
3398,Monica you really think I should try this phas...,Ross,0,576,4,279,14,6,4,"00:14:48,138","00:14:52,390"
